In [19]:
import pandas as pd
import yfinance as yf

In [20]:
df_yahoo = yf.download('AAPL',
                        start='2000-01-01',
                        end='2010-12-31',
                        progress=False)

In [21]:
df_yahoo.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-12-31,0.901228,0.918527,0.888393,0.917969,0.786033,163811200
2000-01-03,0.936384,1.004464,0.907924,0.999442,0.855796,535796800
2000-01-04,0.966518,0.987723,0.903460,0.915179,0.783644,512377600
2000-01-05,0.926339,0.987165,0.919643,0.928571,0.795111,778321600
2000-01-06,0.947545,0.955357,0.848214,0.848214,0.726304,767972800


In [22]:
# 퀀들로 데이터 다운받기

In [23]:
import intrinio_sdk
import pandas as pd
import KEYS

In [24]:
intrinio_sdk.ApiClient().configuration.api_key['api_key'] = KEYS.INTRINIO_KEY
security_api = intrinio_sdk.SecurityApi()

In [25]:
r = security_api.get_security_stock_prices(
    identifier='AAPL',
    start_date='2000-01-01',
    end_date='2010-12-31',
    frequency='daily',
    page_size=10000
)

In [26]:
response_list = [x.to_dict() for x in r.stock_prices]
df_intrinio = pd.DataFrame(response_list).sort_values('date')
df_intrinio.set_index('date', inplace=True)

In [27]:
df_intrinio.head()

,intraperiod,frequency,open,high,low,close,volume,adj_open,adj_high,adj_low,adj_close,adj_volume,factor,split_ratio,dividend,change,percent_change,fifty_two_week_high,fifty_two_week_low
date,,,,,,,,,,,,,,,,,,,
2000-01-03,False,daily,104.87,112.50,101.69,111.94,4783900.0,0.803956,0.862450,0.779578,0.858157,5.357968e+08,1.0,1.0,0.0,0.07,0.0888,0.9,0.25
2000-01-04,False,daily,108.25,110.62,101.19,102.50,4574800.0,0.829868,0.848037,0.775745,0.785787,5.123776e+08,1.0,1.0,0.0,-0.07,-0.0843,0.9,0.25
2000-01-05,False,daily,103.75,110.56,103.00,104.00,6949300.0,0.795370,0.847577,0.789621,0.797287,7.783216e+08,1.0,1.0,0.0,0.01,0.0146,0.9,0.25
2000-01-06,False,daily,106.12,107.00,95.00,95.00,6856900.0,0.813539,0.820285,0.728291,0.728291,7.679728e+08,1.0,1.0,0.0,-0.07,-0.0865,0.9,0.25
2000-01-07,False,daily,96.50,101.00,95.50,99.50,4113700.0,0.739790,0.774288,0.732124,0.762789,4.607344e+08,1.0,1.0,0.0,0.03,0.0474,0.9,0.25


In [28]:
import pandas as pd
import numpy as np
import yfinance as yf

In [29]:
df = yf.download(
    'AAPL',
    start='2000-01-01',
    end='2010-12-31',
    progress=False
)

df = df.loc[:, ['Adj Close']]
df.rename(columns={'Adj Close':'adj_close'}, inplace=True)

In [30]:
df['simple_rtn'] = df.adj_close.pct_change()
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1)) # shift라는 메소드 사용

In [31]:
import pandas as pd
import quandl
import KEYS

In [32]:
quandlkey = KEYS.QUANDL_KEY
quandl.ApiConfig.api_key = quandlkey

In [33]:
df_all_dates = pd.DataFrame(index=pd.date_range(start='1999-12-31', end='2010-12-31'))
df = df_all_dates.join(df[['adj_close']], how='left').fillna(method='ffill').asfreq('M')

In [34]:
df_cpi = quandl.get(dataset='RATEINF/CPI_USA', start_date='1999-12-01',
    end_date='2010-12-31')
df_cpi.rename(columns={'Value':'cpi'}, inplace=True)

In [35]:
df_cpi.head()

,cpi
Date,
1999-12-31,168.3
2000-01-31,168.8
2000-02-29,169.8
2000-03-31,171.2
2000-04-30,171.3


In [36]:
df_merged = df.join(df_cpi, how='left')
df_merged.head()

,adj_close,cpi
1999-12-31,0.786033,168.3
2000-01-31,0.793200,168.8
2000-02-29,0.876344,169.8
2000-03-31,1.038328,171.2
2000-04-30,0.948496,171.3


In [37]:
df_merged['simple_rtn'] = df_merged.adj_close.pct_change()
df_merged['inflation_rate'] = df_merged.cpi.pct_change()
df_merged.head()

,adj_close,cpi,simple_rtn,inflation_rate
1999-12-31,0.786033,168.3,NaN,NaN
2000-01-31,0.793200,168.8,0.009118,0.002971
2000-02-29,0.876344,169.8,0.104820,0.005924
2000-03-31,1.038328,171.2,0.184842,0.008245
2000-04-30,0.948496,171.3,-0.086517,0.000584


In [38]:
df_merged['real_rtn'] = (df_merged.simple_rtn + 1)/(df_merged.inflation_rate +1) - 1
df_merged.head()

,adj_close,cpi,simple_rtn,inflation_rate,real_rtn
1999-12-31,0.786033,168.3,NaN,NaN,NaN
2000-01-31,0.793200,168.8,0.009118,0.002971,0.006129
2000-02-29,0.876344,169.8,0.104820,0.005924,0.098314
2000-03-31,1.038328,171.2,0.184842,0.008245,0.175152
2000-04-30,0.948496,171.3,-0.086517,0.000584,-0.087050


In [39]:
def realized_volatility(x):
    return np.sqrt(np.sum(x**2))

In [40]:
df_rv = df.groupby(pd.Grouper(freq='M')).apply(realized_volatility)
df_rv.rename(columns={'log_rtn':'rv'}, inplace=True)
df_rv['rv'] = df_rv['rv']*np.sqrt(12)

KeyError: 'rv'

In [41]:
df

,adj_close
1999-12-31,0.786033
2000-01-31,0.793200
2000-02-29,0.876344
2000-03-31,1.038328
2000-04-30,0.948496
...,...
2010-08-31,7.434294
2010-09-30,8.677422
2010-10-31,9.204336
2010-11-30,9.515347
